In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

In [ ]:
data=pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv')

In [ ]:
data

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')

In [ ]:
copy_data=data.copy()

In [ ]:
# np.unique(submission['value'])

In [ ]:
# data.index
# data.loc[0,'F_1_0']

In [ ]:
# copy_data.columns

In [ ]:

columns_index={}
columns_max={}
i=0
for colum in copy_data.columns:
    columns_index[i]=colum
    columns_max[colum]=copy_data[colum].mean(axis=0)
    i+=1
    

In [ ]:
# copy_data.max(axis=0)
# copy_data['F_1_0'].max(axis=0)

In [ ]:
values=np.where(copy_data.isnull()==True)
# print(values)

list_1_max=values[0]
list_2_max=values[1]
for i in range(len(list_1_max)):
#     print(val_1,val_2,columns_index[val_2])
    val_1=list_1_max[i]
    val_2=list_2_max[i]
    colum_name=columns_index[val_2]
#     print(copy_data.loc[val_1,colum_name])
    copy_data.loc[val_1,colum_name]=columns_max[colum_name]
#     print(copy_data.loc[val_1,colum_name])
    
#           print(val_1,val_2)
   
        


In [ ]:
values_2=np.where(np.isfinite(copy_data) == False)
# print(values_2)
list_1_inf=values_2[0]
list_2_inf=values_2[1]
for i in range(len(list_1_inf)):
#     print(val_1,val_2,columns_index[val_2])
    val_1=list_1_inf[i]
    val_2=list_2_inf[i]
    colum_name=columns_index[val_2]
#     print(copy_data.loc[val_1,colum_name])
    copy_data.loc[val_1,colum_name]=columns_max[colum_name]
#     print(copy_data.loc[val_1,colum_name])

In [ ]:
# imp = SimpleImputer(
#         missing_values=np.nan,
#         strategy='std')

# data[:] = imp.fit_transform(data)


In [ ]:
kmeans = KMeans(5)
kmeans.fit(copy_data)

In [ ]:
identified_clusters = kmeans.fit_predict(copy_data)
copy_data['clusters']=identified_clusters
copy_data=copy_data.sort_values('clusters')
copy_data

In [ ]:
list_mean=[]
for i in range(5):
    list_mean.append(copy_data.groupby('clusters').mean().loc[i])


In [ ]:
for i in range(len(list_1_max)):
    
    val_1=list_1_max[i]
    val_2=list_2_max[i]
    if(copy_data.loc[val_1,'clusters']==0):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[0][val_2]
    elif(copy_data.loc[val_1,'clusters']==1):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[1][val_2]
    elif(copy_data.loc[val_1,'clusters']==2):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[2][val_2]
    elif(copy_data.loc[val_1,'clusters']==3):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[3][val_2]
    elif(copy_data.loc[val_1,'clusters']==4):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[4][val_2]

In [ ]:
for i in range(len(list_1_inf)):
    val_1=list_1_inf[i]
    val_2=list_2_inf[i]
    if(copy_data.loc[val_1,'clusters']==0):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[0][val_2]
    elif(copy_data.loc[val_1,'clusters']==1):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[1][val_2]
    elif(copy_data.loc[val_1,'clusters']==2):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[2][val_2]
    elif(copy_data.loc[val_1,'clusters']==3):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[3][val_2]
    elif(copy_data.loc[val_1,'clusters']==4):
        copy_data.loc[val_1,columns_index[val_2]]=list_mean[4][val_2]


In [ ]:

for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = copy_data.loc[row, col]

submission.to_csv('mean_benchmark.csv')